**In the shortened version, the following changes were made:**

1. Function and variable names were changed to use lowercase with underscores, following Python's naming conventions.
2. Comments and print statements were removed to improve readability.
3. Looping constructs were simplified and unnecessary indexing was eliminated.
4. Code formatting was improved to make it more consistent and readable.

In [68]:
import csv
import random
import math

def fract(x, y):
    if y == 0:
        return 0
    return x / y

def load_csv(filename):
    with open(filename, "r") as file:
        lines = csv.reader(file)
        dataset = [list(map(float, row)) for row in lines]
    return dataset

def split_dataset(dataset, split_ratio):
    train_size = int(len(dataset) * split_ratio)
    train_set = []
    copy = list(dataset)
    while len(train_set) < train_size:
        index = random.randrange(len(copy))
        train_set.append(copy.pop(index))
    return [train_set, copy]

def separate_by_class(dataset):
    separated = {}
    for vector in dataset:
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = []
        separated[class_value].append(vector)
    return separated

def mean(numbers):
    return fract(sum(numbers), float(len(numbers)))

def stddev(numbers):
    avg = mean(numbers)
    variance = fract(sum([(x - avg) ** 2 for x in numbers]), float(len(numbers) - 1))
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stddev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = {}
    for class_value, instances in separated.items():
        summaries[class_value] = summarize(instances)
    return summaries

def calculate_probability(x, mean, stddev):
    exponent = math.exp(-fract((x - mean) ** 2, (2 * stddev ** 2)))
    return fract(1, (math.sqrt(2 * math.pi) * stddev)) * exponent

def calculate_class_probabilities(summaries, input_vector):
    probabilities = {}
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = 1
        for i, (mean, stddev) in enumerate(class_summaries):
            x = input_vector[i]
            probabilities[class_value] *= calculate_probability(x, mean, stddev)
    return probabilities

def predict(summaries, input_vector):
    probabilities = calculate_class_probabilities(summaries, input_vector)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

def get_predictions(summaries, test_set):
    predictions = []
    for instance in test_set:
        result = predict(summaries, instance)
        predictions.append(result)
    return predictions

def get_accuracy(test_set, predictions):
    correct = sum(1 for i in range(len(test_set)) if test_set[i][-1] == predictions[i])
    return fract(correct, float(len(test_set))) * 100.0

def naive_bayes():
    filename = 'nbayes.csv'
    split_ratio = 0.8
    dataset = load_csv(filename)
    training_set, test_set = split_dataset(dataset, split_ratio)
    print(f"Split {len(dataset)} rows into train={len(training_set)} and test={len(test_set)} rows")
    print("\nFirst 5 rows of training set:", training_set[:5])
    print("\nFirst 5 rows of test set:", test_set[:5])
    summaries = summarize_by_class(training_set)
    predictions = get_predictions(summaries, test_set)
    accuracy = get_accuracy(test_set, predictions)
    print(f"\nAccuracy: {accuracy}")
    print(f"\nPredictions: {predictions}")

naive_bayes()


Split 15 rows into train=12 and test=3 rows

First 5 rows of training set: [[1.0, 189.0, 60.0, 23.0, 846.0, 30.1, 0.398, 59.0, 1.0], [5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0, 0.0], [10.0, 139.0, 80.0, 0.0, 0.0, 27.1, 1.441, 57.0, 0.0], [6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0, 1.0], [0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0, 1.0]]

First 5 rows of test set: [[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0, 0.0], [8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0, 1.0], [2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0, 1.0]]

Accuracy: 100.0

Predictions: [0.0, 1.0, 1.0]
